# Vector Space Models

This chapter discusses computational models to represent text in vector spaces.

* [Preparation](#Preparation)
* [Bag-of-Words](#Bag-of-Words)
* [Frequency Counts](#Frequency-Counts)
* [Exercise 1](#Exercise-1)
* [TF-IDF](#TF-IDF)
* [Similarity Metrics](#Similarity-Metrics)

## References

* [Vector Space Model](https://en.wikipedia.org/wiki/Vector_space_model)
* [Bag-of-Words Model](https://en.wikipedia.org/wiki/Bag-of-words_model)
* [TF-IDF](https://en.wikipedia.org/wiki/Tf–idf)
* [Euclidean Distance](https://en.wikipedia.org/wiki/Euclidean_distance)
* [Cosine Similarity](https://en.wikipedia.org/wiki/Cosine_similarity)

## Preparation

Download [`aesopfables.json`](https://github.com/emory-courses/computational-linguistics/blob/master/docs/res/aesopfables.json) and read the JSON file.

In [3]:
!pip install requests
import requests

def download(remote_addr: str, local_addr: str):
    r = requests.get(remote_addr)
    fin = open(local_addr, 'wb')
    fin.write(r.content)

* [`requests`](https://requests.readthedocs.io/en/master/user/quickstart/)

In [4]:
aesop_link = 'https://raw.githubusercontent.com/emory-courses/computational-linguistics/master/docs/res/aesopfables.json'
aesop_file = 'res/aesopfables.json'
download(aesop_link, aesop_file)

* Make sure which directory `aesopfables.json` is downloaded.

In [11]:
import json
fables = json.load(open(aesop_file))

print(len(fables))
for fable in fables[:10]: print(fable['title'])

357
Androcles
The Ant and the Chrysalis
The Ant and the Dove
The Ants and the Grasshopper
The Apes and the Two Travelers
The Ass and His Driver
The Ass and His Masters
The Ass and His Purchaser
The Ass and His Shadow
The Ass and the Charger


## Bag-of-Words

Let there be a giant bag that can hold all unique words in the world.
Then, each token in a text such as "*Jinho Choi is a professor at Emory University .*" can be inserted to the bag as follows:

<img src="res/bow.jpg" width=600 align="left"/>

This bag can be represented by a vector of which every dimension stands for a unique token in the world.
All dimensions are initialized to `0`, except for the ones representing tokens in the input text, which are assigned with `1`:

<img src="res/vsm.jpg" width=600 align="left"/>

* What is the total dimension of this vector?
* Does this vector correctly represent the original text (anything missing)?

A bag-of-words can be implemented by a dictionary (representing a sparse vector), where the key is a term in the text and its value is always `1`.
The value of every other term that does not appear in the document is assumed to be `0`.

In [6]:
v = {'Jinho': 1, 'Choi': 1, 'is': 1, 'a': 1, 'professor': 1, 'at': 1, 'Emory': 1, 'University': 1, '.': 1}

## Frequency Counts

Consider the following two documents:

```
D1: John bought a car . The car was fancy .
D2: Mary liked the car .  John gave it to Mary .
```

A **term frequency** (`tf`) is the number of occurrences of a specific term in a specific document:

```
tf(John, D1) = 1
tf(John, D2) = 1
tf(Mary, D2) = 2
```

A **document frequency** (`df`) is the number of documents containing a specific term:

```
df(John) = 2
df(John) = 2
```

Let us define the function `term_frequencies()` that takes `fables` above and returns a dictionary where each key-value pair represents the source and term frequencies of the corresponding document:

In [12]:
from collections import Counter
from typing import Dict

def term_frequencies(fables) -> Dict[str, Counter]:
    def key(t): return t[t.rfind('&')+1:]
    return {key(fable['source']): Counter(fable['tokens'].split()) for fable in fables}

* [`collections.Counter`](https://docs.python.org/3/library/collections.html#collections.Counter)

In [13]:
tfs = term_frequencies(fables)
print(tfs['Androcles'])

Counter({'the': 20, 'and': 19, ',': 18, 'to': 13, '.': 12, 'his': 9, 'Lion': 9, 'Androcles': 8, 'was': 8, 'he': 7, 'him': 6, 'came': 4, 'a': 4, 'slave': 3, 'from': 3, 'out': 3, 'all': 3, 'of': 3, 'forest': 2, 'As': 2, 'upon': 2, 'turned': 2, 'that': 2, 'up': 2, 'near': 2, 'paw': 2, 'which': 2, 'thorn': 2, 'had': 2, 'into': 2, 'who': 2, 'soon': 2, 'like': 2, 'dog': 2, 'But': 2, 'The': 2, 'Emperor': 2, 'let': 2, 'loose': 2, 'as': 2, 'A': 1, 'named': 1, 'once': 1, 'escaped': 1, 'master': 1, 'fled': 1, 'wandering': 1, 'about': 1, 'there': 1, 'lying': 1, 'down': 1, 'moaning': 1, 'groaning': 1, 'At': 1, 'first': 1, 'flee': 1, 'but': 1, 'finding': 1, 'did': 1, 'not': 1, 'pursue': 1, 'back': 1, 'went': 1, 'put': 1, 'swollen': 1, 'bleeding': 1, 'found': 1, 'huge': 1, 'got': 1, 'it': 1, 'causing': 1, 'pain': 1, 'He': 1, 'pulled': 1, 'bound': 1, 'able': 1, 'rise': 1, 'lick': 1, 'hand': 1, 'Then': 1, 'took': 1, 'cave': 1, 'every': 1, 'day': 1, 'used': 1, 'bring': 1, 'meat': 1, 'live': 1, 'shortly'

## Exercise 1

Let us define the function `document_frequencies()` that takes `fables` and returns a dictionary where each key-value pair represents a term and its document frequency:

```python
def document_frequencies(fables) -> Dict[str, int]:
    # To be filled
    return dict()
```

In [14]:
dfs = document_frequencies(fables)
print(dfs['Lion'], dfs['lion'])
for term, count in sorted(dfs.items(), key=lambda x: x[1], reverse=True)[:10]:
    print(term, count)

43 5
. 356
, 355
the 350
and 344
to 337
a 335
of 313
" 280
in 275
his 252


* [sorted()](https://docs.python.org/3/library/functions.html?highlight=sorted#sorted)
* [lambda](https://docs.python.org/3/reference/expressions.html#lambda)

## TF-IDF

What are important terms in a document?

* High term frequency
* low document frequency

The $\mathrm{tf}\cdot\mathrm{idf}_{t,d}$ (Term Frequency - Inverse Document Frequency) of a specific term $t$ in a specific document $d \in D$ is measured as follows:

$$
\mathrm{tf}\cdot\mathrm{idf}_{t,d} = \mathrm{tf}_{t,d} \times \log\frac{|D|}{\mathrm{df}_t}
$$

Several variations of $\mathrm{tf}_{t,d}$ have also been proposed using sublinear or normalization:

$$
\begin{align}
\mathrm{wf}_{t,d} &=& \left\{
\begin{array}{cl}
 1 + \log\mathrm{tf}_{t,d} & \mbox{if $\textrm{tf}_{t,d} > 0$}\\
 0 & \mbox{otherwise}
\end{array}
\right. \\
\mathrm{ntf}_{t,d} &=& \alpha + (1-\alpha)\frac{\mathrm{tf}_{t,d}}{\mathrm{tf}_{\mathrm{argmax}({\mathrm{tf}_{\forall t, d}}),d}}
\end{align}
$$


Let us define the function `tf_idfs()` that takes `fables` and returns a dictionary where the key is a (term, document ID) pair, and the value is its TF-IDF score:

In [24]:
from typing import Tuple
import math

def tf_idfs(fables) -> Dict[str, Dict[str, int]]:
    tfs = term_frequencies(fables)
    dfs = document_frequencies(fables)
    out = dict()
    D = len(tfs)

    for dkey, term_counts in tfs.items():
        out[dkey] = {t: tf  / dfs[t] for t, tf in term_counts.items()}

    return out

In [25]:
tfidfs = tf_idfs(fables)
print(tfidfs['Androcles']['Lion'])

0.20930232558139536


In [26]:
for t, score in sorted(tfs['Androcles'].items(), key=lambda x: x[1], reverse=True)[:20]:
    print(t, score)

the 20
and 19
, 18
to 13
. 12
his 9
Lion 9
Androcles 8
was 8
he 7
him 6
came 4
a 4
slave 3
from 3
out 3
all 3
of 3
forest 2
As 2


In [27]:
for t, score in sorted(tfidfs['Androcles'].items(), key=lambda x: x[1], reverse=True)[:20]:
    print(t, score)

Androcles 8.0
Emperor 2.0
wandering 1.0
moaning 1.0
flee 1.0
bleeding 1.0
sentenced 1.0
arena 1.0
bounding 1.0
recognised 1.0
fawned 1.0
licked 1.0
story 1.0
Whereupon 1.0
pardoned 1.0
freed 1.0
slave 0.75
paw 0.6666666666666666
thorn 0.6666666666666666
pursue 0.5


## Similarity Metrics

Given two vectors, $X_i = (x_{i1}, \ldots, x_{in})$ and $X_j = (x_{j1}, \ldots, x_{jn})$, the **Euclidean distance** between $X_i$ and $X_j$ measures the magnitude between them:

$$
\mathrm{Euclidean}(X_i, X_j) = \lVert X_i - X_j \rVert = \sqrt{\sum_{k=1}^n (x_{ik} - x_{jk})^2}
$$

On the other hand, the **Cosine similarity** measures the angle difference between them:


$$
\mathrm{Cosine}(X_i, X_j) = \frac{X_i\cdot X_j}{\lVert X_i\rVert\lVert X_j\rVert} = \frac{\sum_{\forall k}(x_{ik} \cdot x_{jk})}{\sqrt{\sum_{\forall k}(x_{ik})^2} \cdot \sqrt{\sum_{\forall k}(x_{jk})^2}}
$$

<img src="res/vector_similarities.jpg" width=350/>

Let us define the function `euclidean(x1, x2)` that takes two sparse vectors, `x1` and `x2`, and returns their Euclidean distance: